Locality Sensitive Hashing (LSH) 의 실습을 위하여 50 차원 크기의 인공데이터 100 만개를 만듦니다.

In [1]:
import numpy as np

x = np.random.random_sample((1000000, 50))
x.shape

(1000000, 50)

LSHForest 는 Cosine distance 를 이용하는 LSH 입니다 (min-hash). 빠른 검색을 위하여 hash table 의 개수는 4 개로 하였습니다.

In [2]:
%%time
from sklearn.neighbors import LSHForest

lsh = LSHForest(n_estimators=4)
lsh.fit(x)

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/sklearn/neighbors/approximate.py:220: DeprecationWarning: LSHForest has poor performance and has been deprecated in 0.19. It will be removed in version 0.21.
  DeprecationWarning)


CPU times: user 2.44 s, sys: 200 ms, total: 2.64 s
Wall time: 2.64 s


하나의 query vector의 최인접 이웃 10개를 검색합니다. Return 은 distance vector와 각 distance에 해당하는 x의 row id가 출력됩니다. query vector는 x의 33번째 row를 선택하였으며, 최인접이웃 10개를 선택하였습니다. 

`sklearn == 0.18` 에서는 아래와 같이 query 를 넣어도 작동합니다. 단, warning 이 뜰 수 있습니다. 

```python
dist, idxs = lsh.kneighbors(x[33,:], n_neighbors=10)
```

하지만 `sklearn >= 0.19.1` 에서는 reshape 을 반드시 해줘야 합니다. 

In [3]:
%%time
dist, idxs = lsh.kneighbors(x[33,:].reshape(1,-1), n_neighbors=10)

CPU times: user 32 ms, sys: 0 ns, total: 32 ms
Wall time: 17.9 ms


query vector 와 동일한 벡터, 33 번이 검색되었음을 확인했습니다. dist 가 list of list 인 이유는 한 번에 여러 개의 query vectors 에 대하여 최인접이웃을 검색할 수 있기 때문입니다. 10개의 query vector 에 대하여 5 개의 neighbors 를 구하겠습니다.

In [4]:
print(dist)
print(idxs)

[[0.         0.11306707 0.11733111 0.11913376 0.11927383 0.11931782
  0.12011241 0.12057217 0.12205236 0.12256523]]
[[    33 353025 362946 217493 311867 187656  86836 793379 918769 507252]]


kneighbors 는 하나의 row 에 대하여 최인접 이웃을 계산할 수도 있지만, 여러 개의 query points 에 대해서도 최인접 이웃을 찾을 수도 있습니다. 그렇기 때문에 dist 의 type 이 list of list 입니다. 

이번에는 0 ~ 9까지 10개의 rows에 대하여 각각 이웃을 5개씩 찾아봅니다. 

그 결과 dist 의 shape 은 (10, 5) 임을 알 수 있습니다. dist 의 각 row 는 query point 에 해당하고, 최인접이웃에 대하여 각각 row 마다 5 개의 distance 가 계산된 것입니다. 

In [5]:
%%time
dist, idxs = lsh.kneighbors(x[:10,], n_neighbors=5)
print(dist.shape)

(10, 5)
CPU times: user 44 ms, sys: 528 ms, total: 572 ms
Wall time: 72.3 ms


10 개의 query vector 에 대하여 full search 로 최인접이웃을 찾을 때의 거리 계산 비용을 알아봅니다. 이를 통하여 full search 대비 LSH 가 얼마나 효율적으로 최인접 이웃을 찾는지 비교해 봅니다. 

full search 를 하였긔 때문에 dist_full 의 크기는 (1000000, 10) 입니다. 이로부터 최인접이웃 5 개를 선택하기 위하여 sorting 을 수행합니다.

In [6]:
%%time

from sklearn.metrics import pairwise_distances
dist_full = pairwise_distances(x, x[:10])
print(dist_full.shape)

(1000000, 10)
CPU times: user 216 ms, sys: 536 ms, total: 752 ms
Wall time: 235 ms


LSHForest 는 pickling 을 통하여 저장할 수도 있습니다. 결과는 학습했던 lsh 와 동일함을 확인할 수 있습니다. 

In [7]:
import pickle

with open('test_lsh.pkl', 'wb') as f:
    pickle.dump(lsh, f)
    
with open('test_lsh.pkl', 'rb') as f:
    loaded_lsh = pickle.load(f)
    
loaded_lsh.kneighbors(x[0,:].reshape(1,-1))

(array([[0.        , 0.08934844, 0.10296299, 0.10518009, 0.10554744]]),
 array([[     0, 114315, 230750,  94896, 713117]]))